In [2]:
from torch import nn
import torch
from torch.utils import data

In [3]:
# 定义网络模型
# Sequential就是对所有网络层的一个包裹，让其形成一个流水线
"""
对于标准深度学习模型，我们可以使⽤框架的预定义好的层。这使我们只需关注使⽤哪些层来构造模型，⽽
不必关注层的实现细节。我们⾸先定义⼀个模型变量net，它是⼀个Sequential类的实例。 Sequential类将多
个层串联在⼀起。当给定输⼊数据时， Sequential实例将数据传⼊到第⼀层，然后将第⼀层的输出作为第⼆
层的输⼊，以此类推。在下⾯的例⼦中，我们的模型只包含⼀个层，因此实际上不需要Sequential。但是由
于以后⼏乎所有的模型都是多层的，在这⾥使⽤Sequential会让你熟悉“标准的流⽔线”
"""
## nn.Linear代表全连接，也就是WX，2代表该层的输入，1代表该层的输出
net = nn.Sequential(nn.Linear(2, 1))

In [4]:
# 初始化网络模型参数
"""
如在线性回归模型中的权重和偏置，深度学习框架通常有预定义的⽅法来初始化参数。
在这⾥，我们指定每个权重参数应该从均值为0、标准差为0.01的正态分布中随机采样，偏置参数将初始化为零
"""
"""
通过net[0]选择⽹络中的第⼀个图层，然后使⽤weight.data和bias.data⽅法访问参数。net[0]表示第一层网络
还可以使⽤替换⽅法normal_和fill_来重写参数值。
"""
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [5]:
# 定义损失函数
loss = nn.MSELoss()

In [7]:
# 定义优化算法
"""
⼩批量随机梯度下降算法是⼀种优化神经⽹络的标准⼯具， PyTorch在optim模块中实现了该算法的许多变种。
当我们实例化⼀个SGD实例时，我们要指定优化的参数（可通过net.parameters()从我们的模型中获得）
以及优化算法所需的超参数字典。⼩批量随机梯度下降只需要设置lr值，这⾥设置为0.03
"""
trainer = torch.optim.SGD(net.parameters(), lr=0.03)
print(*net.parameters())#因为只有一层网络，所以就是W和b两个参数

Parameter containing:
tensor([[-0.0112,  0.0270]], requires_grad=True) Parameter containing:
tensor([0.], requires_grad=True)


In [17]:
# 生成数据
def synthetic_data(w, b, num_examples):
    """⽣成y=Xw+b+噪声"""
    # x为标准状态分布，生成的列维度和w一样
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    # 噪声
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))


true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)
data_iter = data.DataLoader(data.TensorDataset(*(features, labels)), batch_size=10, shuffle=True)

In [18]:
# 训练
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()  #清除梯度
        l.backward() # 求导
        trainer.step()  #更新梯度
        l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000329
epoch 2, loss 0.000096
epoch 3, loss 0.000096


In [21]:
print(net[0].weight.data,true_w,sep="——")
print(net[0].bias.data,true_b,sep="——")

tensor([[ 1.9998, -3.3987]])===tensor([ 2.0000, -3.4000])
tensor([4.1998])===4.2
